In [1]:
# Prepare Database as Backend
import sqlite3

# Create a new SQLite database
conn = sqlite3.connect('data/university_chatbot.db')
cursor = conn.cursor()

# Create tables
# Students Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        matriculation_number INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        surname TEXT NOT NULL,
        address TEXT
    )
''')

# Courses Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT NOT NULL,
        instructor TEXT NOT NULL
    )
''')

# Registrations Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS registrations (
        registration_id INTEGER PRIMARY KEY,
        matriculation_number INTEGER NOT NULL,
        course_id INTEGER NOT NULL,
        FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
        FOREIGN KEY (course_id) REFERENCES courses (course_id)
    )
''')

# Exam Results Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS exam_results (
        result_id INTEGER PRIMARY KEY,
        matriculation_number INTEGER NOT NULL,
        course_id INTEGER NOT NULL,
        grade TEXT NOT NULL,
        FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
        FOREIGN KEY (course_id) REFERENCES courses (course_id)
    )
''')

# Commit changes and close the connection
conn.commit()
conn.close()


In [2]:
# Populate Database with data

import random

# Connect to the SQLite database
conn = sqlite3.connect('data/university_chatbot.db')
cursor = conn.cursor()

# Sample data for students
students = [
    (201001, 'Alice', 'Smith', '123 Park Ave'),
    (201002, 'Bob', 'Johnson', '456 Elm St'),
    (201003, 'Carol', 'Williams', '789 Oak St'),
    (201004, 'David', 'Brown', '101 Maple Dr'),
    (201005, 'Eve', 'Davis', '202 Pine Rd'),
    (201006, 'Frank', 'Miller', '303 Birch Blvd'),
    (201007, 'Grace', 'Wilson', '404 Cedar Ln'),
    (201008, 'Henry', 'Moore', '505 Spruce Ct'),
    (201009, 'Ivy', 'Taylor', '606 Aspen Way'),
    (201010, 'Jack', 'Anderson', '707 Walnut St')
]

# Sample data for courses
courses = [
    (101, 'Introduction to Computer Science', 'Dr. John Doe'),
    (102, 'Advanced Mathematics', 'Dr. Jane Smith'),
    (103, 'Physics for Engineers', 'Dr. Emily Johnson')
]

# Insert students into the database
cursor.executemany('INSERT INTO students VALUES (?,?,?,?)', students)

# Insert courses into the database
cursor.executemany('INSERT INTO courses VALUES (?,?,?)', courses)

# Generate registrations (2 for each student)
registrations = []
for student in students:
    registered_courses = random.sample(courses, 2)  # Randomly pick 2 courses for each student
    for course in registered_courses:
        registrations.append((None, student[0], course[0]))

# Insert registrations into the database
cursor.executemany('INSERT INTO registrations (registration_id, matriculation_number, course_id) VALUES (?,?,?)', registrations)

# Generate exam results (1 for each student)
exam_results = []
for student in students:
    course_id = random.choice(registrations)[2]  # Randomly pick one course from registrations
    grade = random.choice(['A', 'B', 'C', 'D', 'E', 'F'])  # Randomly assign a grade
    exam_results.append((None, student[0], course_id, grade))

# Insert exam results into the database
cursor.executemany('INSERT INTO exam_results (result_id, matriculation_number, course_id, grade) VALUES (?,?,?,?)', exam_results)

# Commit changes and close the connection
conn.commit()
conn.close()

In [3]:
#Importing nltk + packages
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/ml/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ml/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def process_input(user_input):
    tokens = word_tokenize(user_input)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

# Example usage
sample_input = "I would like to change my address because I have moved out."
processed_input = process_input(sample_input)
print(processed_input)


['would', 'like', 'change', 'address', 'moved']


In [9]:
def identify_intent(processed_input):
    intent_keywords = {
        'register_exam': ['register', 'exam'],
        'change_address': ['change', 'address'],
        # Add more intents and associated keywords here
    }

    for intent, keywords in intent_keywords.items():
        if all(keyword in processed_input for keyword in keywords):
            return intent

    return 'unknown_intent'

# Example usage
intent = identify_intent(processed_input)


In [14]:
import re
def has_postcode(user_input):
    postcode_pattern = re.compile(r'\b\d{5}\b')
    return bool(re.search(postcode_pattern, user_input))
def identify_yes_no_answer(input):
    intent_keywords = {
        'yes': ['true', 'yes','yeah', 'yep'],
        'no': ['wrong', 'false', 'no', 'not', 'nope', 'nah'],
        # Add more intents and associated keywords here
    }

    for intent, keywords in intent_keywords.items():
        if any(keyword in input for keyword in keywords):
                return intent

    return 'unknown_intent'
if intent == 'change_address':
    #Ask for student's matriculation number
    matriculation_number = input("Please enter your matriculation number: ")
    #Connect to the SQLite database
    conn = sqlite3.connect('data/university_chatbot.db')
    cursor = conn.cursor()
    #Check if student exists
    cursor.execute('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
    student = cursor.fetchone()
    if student is None:
        print("Sorry, you are not registered as a student.")
    else:
        #Ask for new address
        new_address = input("Please enter your new address ")
        while not has_postcode(new_address):
            postal_code = input("Can you give me your postal code please? ")
            new_address = new_address + " " + postal_code
        
        answer = input("Let me summerize once again: You have moved out and your new address is " + new_address + "? ")
        processed_answer = identify_yes_no_answer(answer)
        while processed_answer == 'unknown_intent':
            answer = input("Sorry, I didn't understand. Please answer with yes or no. ")
            processed_answer = identify_yes_no_answer(answer)
        if processed_answer == 'no':
            new_address = input("Please enter your new address ")
            while not has_postcode(new_address):
                postal_code = input("Can you give me your postal code please? ")
                new_address = new_address + " " + postal_code

        #Update student's address
        cursor.execute('UPDATE students SET address=? WHERE matriculation_number=?', (new_address, matriculation_number))
        conn.commit()
        print("Your address has been updated.")
    #Close the connection
    conn.close()
    

test ade
Your address has been updated.
